In [2]:
import threading

def print_cube(num):
    # function to print cube of given num
    print("Cube: {}".format(num ** 3))

def print_square(num):
    # function to print square of given num
    print("Square: {}".format(num ** 2))

if __name__ == "__main__":
    # creating threads
    t1 = threading.Thread(target=print_square, args=(10,))
    t2 = threading.Thread(target=print_cube, args=(10,))

    # start the threads
    t1.start()
    t2.start()

    # wait until both threads finish
    t1.join()
    t2.join()

    # continue with the main loop
    for i in range(5):
        print("Main loop: {}".format(i))

    print("Done!")


Square: 100
Cube: 1000
Main loop: 0
Main loop: 1
Main loop: 2
Main loop: 3
Main loop: 4
Done!


In [1]:
import threading

def my_function(arg):
    # Công việc của luồng
    print("Đối số là:", arg)
    print("Đây là luồng đang chạy!!")

# Tạo luồng và truyền tham số
my_thread = threading.Thread(
    target=my_function,
    args=(10,)      # args phải là tuple
)

# Bắt đầu chạy luồng
my_thread.start()

# (Không bắt buộc) chờ luồng kết thúc
my_thread.join()


Đối số là: 10
Đây là luồng đang chạy!!


In [3]:
import threading

class PrintThread(threading.Thread):
    def __init__(self, num):
        super().__init__()
        self.num = num

    def run(self):
        # Code chạy trong luồng
        self.print_square()
        self.print_cube()

    def print_cube(self):
        # function to print cube of given num
        print("Cube: {}".format(self.num ** 3))

    def print_square(self):
        # function to print square of given num
        print("Square: {}".format(self.num ** 2))


if __name__ == "__main__":
    # creating thread
    t = PrintThread(10)

    # start the thread
    t.start()

    # wait until the thread finishes
    t.join()

    # continue with the main loop
    for i in range(5):
        print("Main loop: {}".format(i))

    print("Done!")


Square: 100
Cube: 1000
Main loop: 0
Main loop: 1
Main loop: 2
Main loop: 3
Main loop: 4
Done!


In [4]:
# A) Đọc file tuần tự (single-thread)

import time
import pandas as pd

# Danh sách các file cần đọc
files = ["sales_jan.csv", "sales_feb.csv", "sales_mar.csv"]

def read_file_sequential():
    """Hàm đọc tuần tự các file CSV trong danh sách files."""
    start = time.time()

    for f in files:
        time.sleep(0.2)        # mô phỏng thao tác I/O chậm
        df = pd.read_csv(f)    # đọc file
        print(f"Đọc xong {f}: {len(df)} dòng")

    end = time.time()
    print(f"Thời gian đọc tuần tự: {end - start:.4f} giây")


# Gọi hàm đọc
read_file_sequential()


Đọc xong sales_jan.csv: 31 dòng
Đọc xong sales_feb.csv: 29 dòng
Đọc xong sales_mar.csv: 31 dòng
Thời gian đọc tuần tự: 0.6642 giây


In [5]:
# B) Đọc file bằng threading.Thread

import threading
import time
import pandas as pd

results = []     # nơi lưu kết quả đọc file
files = ["sales_jan.csv", "sales_feb.csv", "sales_mar.csv"]

def load_csv_thread(filename):
    """Hàm chạy trong từng luồng để đọc một file CSV"""
    time.sleep(0.2)       # mô phỏng I/O chậm
    df = pd.read_csv(filename)
    
    # Lưu kết quả
    results.append((filename, len(df)))
    
    print(f"Đọc xong {filename}: {len(df)} dòng")

def read_file_multithread():
    start = time.time()

    threads = []         # danh sách chứa các luồng

    # Tạo các luồng
    for f in files:
        t = threading.Thread(target=load_csv_thread, args=(f,))
        threads.append(t)
        t.start()

    # Chờ tất cả luồng kết thúc
    for t in threads:
        t.join()

    end = time.time()
    print(f"Thời gian đọc đa luồng: {end - start:.4f} giây")


# Chạy hàm
read_file_multithread()

# Xem lại kết quả
print("\nKết quả đọc file:")
for name, rows in results:
    print(f"{name}: {rows} dòng")


Đọc xong sales_jan.csv: 31 dòngĐọc xong sales_feb.csv: 29 dòng
Đọc xong sales_mar.csv: 31 dòng

Thời gian đọc đa luồng: 0.2468 giây

Kết quả đọc file:
sales_jan.csv: 31 dòng
sales_feb.csv: 29 dòng
sales_mar.csv: 31 dòng


In [6]:
import threading

# Biến toàn cục x
x = 0

# Hàm increment(): Tăng giá trị biến toàn cục x lên 1
def increment():
    # Cần khai báo 'global x' để sửa đổi biến toàn cục x
    global x
    x += 1 # Thao tác này KHÔNG phải là nguyên tử (atomic), dẫn đến Race Condition

# Hàm thread_task(): Thực hiện gọi hàm increment() 100000 lần
def thread_task():
    '''
    Thực hiện tác vụ lặp, gọi hàm increment() 100000 lần.
    '''
    for _ in range(100000):
        increment()

# Hàm main_task(): Thực hiện nhiệm vụ chính của chương trình
def main_task():
    # Đặt lại giá trị của biến toàn cục x = 0 trước khi tạo luồng mới
    global x
    x = 0

    # Tạo hai luồng (t1 và t2) cùng thực hiện hàm thread_task
    t1 = threading.Thread(target=thread_task)
    t2 = threading.Thread(target=thread_task)

    # Bắt đầu chạy các luồng
    t1.start()
    t2.start()

    # Chờ cho đến khi cả hai luồng (t1 và t2) hoàn thành công việc
    t1.join()
    t2.join()

    # Sau khi cả hai luồng kết thúc, giá trị cuối cùng của x đã được cập nhật
    # Giá trị kỳ vọng là 2 * 100000 = 200000,
    # nhưng do Race Condition, giá trị thực tế có thể nhỏ hơn.


# Chương trình chạy vòng lặp 10 lần, mỗi lần gọi main_task()
print("Chương trình minh họa Race Condition:")
print("-" * 40)
for i in range(10):
    main_task()
    # Sau đó in giá trị x ở mỗi lần lặp ra màn hình.
    print(f"Lần lặp {i+1}: x = {x}")

print("-" * 40)
print("Giá trị kỳ vọng của x là 200000.")
print("Do Race Condition, giá trị thực tế thường nhỏ hơn giá trị kỳ vọng.")

Chương trình minh họa Race Condition:
----------------------------------------
Lần lặp 1: x = 200000
Lần lặp 2: x = 200000
Lần lặp 3: x = 200000
Lần lặp 4: x = 200000
Lần lặp 5: x = 200000
Lần lặp 6: x = 200000
Lần lặp 7: x = 200000
Lần lặp 8: x = 200000
Lần lặp 9: x = 200000
Lần lặp 10: x = 200000
----------------------------------------
Giá trị kỳ vọng của x là 200000.
Do Race Condition, giá trị thực tế thường nhỏ hơn giá trị kỳ vọng.


In [7]:
import threading
import time

# 1. Tạo một Lock (Khóa)
# Lock là cơ chế đồng bộ hóa cơ bản. Chỉ một luồng có thể giữ Lock tại một thời điểm.
lock = threading.Lock()

# 2. Biến toàn cục (Tài nguyên chia sẻ)
balance = 100

# Hàm thực hiện việc thêm tiền một cách an toàn (Safe Increment)
def add_money_safe():
    global balance
    
    # Yêu cầu khóa. Chỉ 1 luồng được cấp quyền đi qua lệnh này.
    # Các luồng khác sẽ bị chặn (block) tại đây cho đến khi Lock được nhả ra.
    lock.acquire()
    
    try:
        # --- Vùng mã Critical Section (CS) ---
        # Mã trong khối này được bảo vệ bởi Lock.
        # Luôn luôn đảm bảo rằng chỉ CÓ MỘT luồng được thực thi code tại đây.
        
        tmp = balance
        # Mô phỏng độ trễ (delay) hoặc các tác vụ tính toán phức tạp.
        # Độ trễ này là nguyên nhân chính gây ra Race Condition nếu không có Lock.
        time.sleep(0.001) 
        balance = tmp + 10
        
    finally:
        # Đảm bảo khóa luôn được nhả ra (release) ngay cả khi có lỗi xảy ra
        # để luồng khác có thể giành được quyền truy cập.
        lock.release() 
        # --- Kết thúc Critical Section ---

# 3. Tạo và chạy các luồng
# Tạo 2 luồng, cả 2 đều chạy cùng một hàm add_money_safe()
threads = [threading.Thread(target=add_money_safe) for _ in range(2)]

print(f"Giá trị ban đầu của balance: {balance}")

# Cả 2 luồng bắt đầu chạy gần như cùng lúc
for t in threads:
    t.start()

# Chờ cho đến khi cả hai luồng hoàn thành công việc
for t in threads:
    t.join()

# 4. In kết quả cuối cùng
# Mỗi luồng cộng 10. Tổng cộng 2 luồng cộng 20.
# Giá trị kỳ vọng là 100 + 20 = 120.
print(f"Giá trị cuối cùng của balance: {balance}")
print("-" * 50)
print("Kết quả: Giá trị cuối cùng LUÔN LUÔN là 120.")
print("Điều này chứng tỏ Lock đã đồng bộ hóa truy cập, loại bỏ Race Condition.")

Giá trị ban đầu của balance: 100
Giá trị cuối cùng của balance: 120
--------------------------------------------------
Kết quả: Giá trị cuối cùng LUÔN LUÔN là 120.
Điều này chứng tỏ Lock đã đồng bộ hóa truy cập, loại bỏ Race Condition.


In [8]:
import threading
import time

# Tạo một Lock (Khóa)
lock = threading.Lock()

# Biến toàn cục (Tài nguyên chia sẻ)
balance = 100

# Hàm thực hiện việc thêm tiền một cách an toàn
def add_money_safe():
    global balance
    
    # 1. Yêu cầu khóa. Chỉ 1 luồng được cấp quyền đi qua.
    lock.acquire()
    
    try:
        # Vùng mã CS (Critical Section): Mã được bảo vệ
        tmp = balance
        # Mô phỏng độ trễ (delay)
        time.sleep(0.001) 
        balance = tmp + 10
        
    finally:
        # 2. Nhả khóa, cho phép luồng khác giành quyền truy cập
        lock.release() 

# Tạo 2 luồng, cả 2 đều chạy cùng một hàm add_money_safe()
threads = [threading.Thread(target=add_money_safe) for _ in range(2)]

print(f"Giá trị ban đầu của balance: {balance}")

# Bắt đầu chạy các luồng
for t in threads:
    t.start()

# Chờ cho đến khi cả hai luồng hoàn thành
for t in threads:
    t.join()

# In kết quả cuối cùng: 100 + 10 + 10 = 120 (luôn chính xác)
print(f"Giá trị cuối cùng của balance: {balance}")
print("Lock đảm bảo rằng mỗi thao tác cộng 10 được thực hiện hoàn toàn mà không bị gián đoạn.")

Giá trị ban đầu của balance: 100
Giá trị cuối cùng của balance: 120
Lock đảm bảo rằng mỗi thao tác cộng 10 được thực hiện hoàn toàn mà không bị gián đoạn.
